In [16]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import combinations
from collections import Counter, defaultdict
import re

In [17]:
#define keywords
keywords = {
    "Arts and Sciences": ["arts and science", "letters and science", "arts, sciences, and letters", "arts, sciences", "arts & sciences", "letters, sciences", "arts and letters", "arts & letters"],
    "Engineering": ["engineering", "mines", "aerospace", "aeronautics", "polytechnic studies", "aviation", "applied science", "computational science"],
    
    "Computer Science": ["computer science", "computer,", "software engineering", "and computer", "computing"],
    "Data Science": ["data science", "artificial intelligence", "machine learning"],
    "Technology": ["technology"],
    "Information": ["information science", "information management", "information systems", "information stud", "library science", "and information", "library stud", "informatics", "school of information", 
                    "information and", "information school"],
    "Archictecure": ["architecture", "architectural", "urban design", "urban planning", "environmental design", "environment and design", "design, construction", "school of construction"],
    

    "Behavioral Science": ["psychology", "psychological", "human science", "human ecology", "behavioral", "human and", "childhood stud"],
    "Social Sciences": ["social science", "social and", "social ecology"],
    "Natural Sciences": ['natural science', "mathematics", "chemistry", "biology", "mathematical", "physical science", "life science", "biological science", 
                        "biochemistry", "natural and", "basic", "college of science", "school of science", ", science, and", ", science and", "physics", "college of the sciences"],
    "Environmental and Earth Science": ["environmental science", "forestry", "natural resource", "college of forest", 
                              "school of forest", "school of environment ", "environmental and", "geology", "oceanography", "earth science", "geophysics", "seismology", "volcanology", "earth and" 
                              "paleontology", "mineral science", "geoscience", "earth science", "earth studies", "atmospheric", "college of the environment", "school of ecology", "school of environment"],
    "Marine Science": ["marine science", "marine stud", "ocean science", "oceanography", "oceanographic", "ocean and"],

    
    "Business": ["business", "commerce", "mba", "college of economics", "and economics", "finance", "entrepreneurship", "school of management", "and management", "accounting", "agribusiness", "economic,", "accountancy", "schools of management", "college of management"],
    "Hospitality": ["hospitality", "hotel administration", "hotel management", "restaurant management"],
    "Education": ["college of education", "school of education", "teaching", "teach", "curriculum development", "human development", "and education", ", education and"],
    "Criminal Justice": ["criminal justice", "criminology", "criminal", "forensics"],


    "Law": ["law", "legal studies"],
    "Policy and Affairs": ["public policy", "international policy", "public affairs", "international affairs", "environmental affairs", "urban affairs", "metropolitan affairs", "international service", "leadership studies", 
                           "diplomacy", "diplomat", "international studies", "public administration", "urban policy", "global studies", "international relations", "social policy", "peace studies", "global management", "policy studies",
                           "policy, planning", "school of leadership", "policy science"],
    

    "Agriculture": ["agriculture", "agricultural", "agribusiness"],
    "Family and Consumer Science": ["family and", "home economics", "consumer science", "family science", "home", "family studies"],
    "Culinary": ["culinary", "cooking", "gastronomy"],


    "Journalism": ["journalism"],
    "Communication": ["communication"],
    "Liberal Arts": ["liberal art", "liberal studies", "interdisciplinary studies", "general studies", "liberal and", "individualized study"],
    "Humanities": ["humanities", "history", "historical", "philosophy", "literature", "political"],
    "Fine Arts": ["fine arts", "music", "theater", "creative", "theatre", "film", "performing art", "the arts", "design art", "dramatic arts", "dance",
                   "visual arts", "school of drama", "recording arts", "ceramics", "fine and applied art", "school of design", "art and design", "fine", 
                   "cinema", "art ", "art, ", "studio art", "media art", "arts and performance", "drama", "arts and media", "textiles", "applied arts"],
    "Language and Culture": ["languages", "lingusitics", "cultures", "translational studies", "intercultural studies", "ethnic studies", "hebraic studies", "cultural stud", "asian studies", "culture ", " hawaiian knowledge", 
                             "jewish", "asian and pacific studies"],


    "Nursing": ["nursing"],
    "Physical Therapy and Nutrition": ["physical therapy", "nutrition", "physiotherapy", "nutritional therapy", "rehab", "human performance"],
    "Medical": ["medical", "pharmacy", "surgeons", "school of medicine", "college of medicine", "podiatric medicine", "osteopathic medicine", "human medicine"],
    "Pharmacy": ["pharmacy", "pharmacology"],
    "Dentistry" : ["dentistry", "dental", "denistry"],
    "Optometry" : ["optometry", "optics", "optical science"],
    "Veterinary" : ['veterinary'],
    "Audiology": ["audiology"],
    "Gerontology": ["gerontology"],
    "Pathology": ["pathology"],
    "Health Science": ["health science", "public health", "health profession", "college of health", "school of health", "biomedical science", "and health", "allied health", "physical education",
                        "health and", "health studies", "genome", "occupational therapy"],


    "Religious Studies": ["religious", "theology", "divinity", "biblical", "theological", "religion", "seminary", "religous", "christ college", "talmudic", "torah", "quran", "bible"],
    "Social Work": ["social work", "social welfare", "community service", "human service", "social service", "public service", "social administration"],
    

    "Graduate": ["graduate school", "graduate studies", "graduate college", "college of graduate", "school of graduate",  "advancing", "advanced study", "graduate division", "graduate and professional studies", "doctoral", "graduate study", "graduate "],
    "Continued Studies": ["extended studies", "extended education", "continuing education", "continued education", "continuing studies", "lifelong learning", "professional programs", "extended learning", "adult learning",
                           "adult education", "online learning", "adult student", "lifelong education", "continuing and", "university extension", "distance learning", "continuous education", "extended university"],
    "Professional Studies": ["professional studies", "professional development", "professional education", "school of professions", "professional campus"],
    # "Administration": ["academic service", "academics", "academic program", "admissions", "admission", "enrollment service", "student life", "housing", "academic support", "advising", "enrollment management", 
    #                    "student relations", "academic computing", "academic records", "student service", "student affairs", "student development", "registrar", "financial aid", "media service", "library service", 
    #                    "university librar", "information service", "information access", "library access", "residence", "academic affairs", "library affairs", "academic succes", "first year student",
    #                    "undergraduate studies", "undergraduate education"],
    "Honor College": ["honor college", "honors college", "scholar program"],

    # "Satellite Campus": ["campus"],
    # "Foundation": ["foundation"],
    # "Boards": ["board of", "regents", "trustees", "council of"],
    "Hospital": ["hospital"],
    "University College": ["university college"],
    "Error": []
}

In [ ]:
VP_KEYWORDS = {
    "SubInstitution": ["arts and science", "the college", "the arts", "interdisciplinary studies", "life sciences", "agriculture", "agricultural", "liberal arts", "engineering", "law center"],
    "Generic": [],
    "Foundation": ["foundation"],


    "Administration": ["vice president, administration", "and administration", "administrative", "audit and compliance", "legal affairs", "legal services", "institutional affairs", "university affairs", "vice president, operations", 
                       "employment service", "vice president, quality", "management service", "university compliance" "university operation", "institutional services", "general administration",
                       "of the corporation", "special program", "university governance", "materials management", "executive affair"," university operation", "vice president. administration"],
    "Boards": ["of the board", "board of", "board operation"],
    "Campus Operations": ["sustainability", "facilities", "campus operations", "infrastructure", "environmental", "security", "campus police", "emergency management", "safety", 
                          "campus services", "university services", "facility operation", "university architect", "land, buildings and real estate", "auxiliary", "capital construction",
                          "property management", "plant operations", "construction", "campus residence", "physical resources", "university compliance", "physical plant", "facilities"],

    "Health Affairs": ["health service", "health affair", "ambulatory", "clinical service", "cancer", "patient care", "diagnostic service", "physicians",
               "clinical", "care contracting", "outpatient", "patient service", "medical affair", "health system affairs", "surgical service", "health system",
               "community health", "health policy"],


    "Academic Affairs": ["undergraduate education", "curriculum", "provost" "undergraduate studies", "graduate studies", "instruction", "academic program", "academic development",
                         "education affair", "undergraduate affair", "academie affairs", "educational and", "teacher education", "experiential education", "academic affair", "academic administration",
                         "undergraduate initiative", "cooperative education", "school program", "interdisciplinary education", "educational outreach", "educational,", "undergraduate studies", "educational affair",
                         "academic service", "academic afair", "educational resource", "academic enrichment", "academic resource", "university studies", "vice president, academics", "academic operation",
                         "educational development", "academic issues", "academic planning", "career development", "academic vice president"],
    "Faculty Affairs": ["faculty affair", "staff relations", "vice president, faculty"],
    "Research": ["innovation", "research", "grants", "publication", "biotechnology", "scientific", "sponsored programs"],
    

    "Student Affairs": ["student", "student life", "student service", "support", "wellbeing", "well-being", "campus life", "housing", "counseling", "student affairs", "student initiative", "student service", 
                        "student success", "student activities", "student development", "students", "university life", "student learning", "residence life", "student and community",
                        "career service", "student judicial service", "advisement", "student access", "study abroad", "dining program", "academic life"],
    "Enrollment": ["enrollment", "financial aid", "retention", "registrar", "admission", "enrolment"],
    

    "Information Service": ["technology", "information", "librar", "data management", "computer", "computing", "network service", "internet service", "electronic marketing", "converging technologies",
                    "technical service", "vice president, systems", "enterprise system", "records", "system administration"],


    "DEI": ["diversity", "equity", "inclusion", "affirmative action", "dei", "multicultural affairs", "equal opportunity", "minority affair", "intercultural affair"],
    "External Relations": [ "external", "university relation", "college relation", "public relation", "communications", "public affair", "media", "outreach", "partnership", "global",
                            "international", "study abroad", "federal relations", "public events", "institutional relation",
                           "government", "state relations", "community engagement", "community affair", "civic engagement", "community relation", "corporate relation", 
                           "foundation relation", "legislative", "policy affairs", "defense relation", "constituent relations", "regional affair", "system relation", "regional operation",
                           "federal program", "agency relation", "regional engagement", "communication", "news services", "federal affair", "commonwealth relation", "state affair", "institute relation", "community services",
                           "intercampus", "urban affairs", "public service", "school relation", "public policy", "district of columbia affair", "d.c. affair", "vice president, engagement", "vice president, policy", "institute affair",
                           "community development"],


    "Planning": ["planning", "assessment", "evaluation", "planner", "special projects", "strategy and measurement", "institutional effectiveness", "presidential initiative", "capital improvement", "strategy and policy",
                 "university projects", "institutional analysis", "commercialization strategies", "procurement", "strategic affair", "university initiative", "program development", "strategic initiative", "strategic development",
                 "strategic and"],
    "Finance": ["budget", "finance", "financial", "accounting", "marketing", "endowment",  "investment", "business", "fiscal", "and resource management", "institutional resources", "treasury management", "university resources", "audit", ", resource management",
                "commerce", "asset management"],
    "Advancement": ["fundraise", "fundraising", "alumni", "donation", "parent relation", "annual giving", "special events", "principal gifts", "major gift", "campaigns", 
                    "planned giving", "leadership gifts", "gift program", "trusts and estate", "giving", "philanthropy", "constituent program", "institutional advancement", "vice president, advancement", "vice president, development",
                    "university advancement", "university development", "vice president. development", "vice president. advancement", "economic advancement", "economic development",
                    "college advancement", "institute advancement", "and development", "and advancement", "resource development", "institutional advancement", "sponsored funds", "college development", "wide advancement", "institution advancement",
                    "regional development", "planned gift"],
    "Human Resources": ["hr", "human resource", "personnel", "recruitment", "employee relations", "union", "collective bargaining", "labor", "human relations", "human capital", "personnel", "human and", "professional development", "human service", "hiring",
                        "workforce"],
    

    "Religious": ["ministry", "mission", "ministries", "church", "religious", "religion", "spiritual", "christian development"],
    "Satellite Campus": ["satellite campus"],
    "Athletics": ["athletic", "sports"],

    "Other": []
}

In [19]:
def extract_all_vp(df):
    mask = df['Position'].str.contains("Vice President", case=False, na=False)
    vp_df = df.loc[mask].copy()
    return pd.DataFrame(vp_df)


def check_keyword(row):
    pos = row["Position"].title()
    inst = row["Institution"].title()
    subinst = row["SubInstitution"]
    blank = pd.isna(subinst)

    # Precompute the “vice president” suffix (e.g. “Of Finance” in “Vice President Of Finance”)
    lower = pos.lower()
    idx = lower.find("vice president")
    vp_type = pos[idx + len("vice president"):].strip() if idx >= 0 else ""

    matches = []
    for key, keywords in VP_KEYWORDS.items():
        for kw in keywords:
            if kw.title() in pos and "director, " not in lower:
                matches.append(key)
                break 
    if matches:
        return ", ".join(matches)

    generic_titles = {"Vice President","Executive Vice President","Head Vice President","Chief Vice President",}
    if pos in generic_titles:
        return "Generic" if blank else "SubInstitution"

    #Blank + no suffix + no punctuation = Generic
    if blank and (vp_type == "" or ("," not in pos and "." not in pos)):
        return "Generic"

    #Institution name inside title (but not “Foundation”) = Satellite Campus
    if inst in pos and "Foundation" not in pos:
        return "Satellite Campus"

    return ""


def classify_vp(df):
    out = df.copy()
    out['FixedPosition'] = out.apply(check_keyword, axis=1)
    return out

def remaining_classification(vp_df):
    for index, row in vp_df.iterrows():
        type = row['FixedPosition']
        type_blank = False
        if type == "":
            type_blank = True
        position = row["Position"].title()
        if "Campus" in position and type_blank == True:
            vp_df.at[index, 'FixedPosition'] += "Satellite Campus, "
        if "Academic" in position and type_blank == True:
            vp_df.at[index,'FixedPosition'] += "Academic Affairs, "
        if "Administration" in position and type_blank == True:
            vp_df.at[index, 'FixedPosition'] += "Administration, "
        if ("Development" in position or "Advancement" in position) and type_blank == True:
            vp_df.at[index, 'FixedPosition'] += "Advancement, "
        if type == "" and "Campus" in position:
            vp_df.at[index, 'FixedPosition'] = "Satellite Campus"
        elif type == "":
            vp_df.at[index, 'FixedPosition'] = "Other"
        vp_df.at[index, 'FixedPosition'] = vp_df.at[index, 'FixedPosition'].rstrip(", ")
    return vp_df
        
def assign_position_grouping_subinst(position):
    classification = ""
    for key, value in keywords.items():
        value_upper = [s.title() for s in value]
        keyword_appears = any(s in position for s in value_upper)
        dean_comma = position.rfind("Dean,") + 5
        if key.title() == position[dean_comma:]:
            classification += key + ", "
        elif keyword_appears:
            classification += key + ", "
    return classification.strip(", ")



def classify_subinstitution_vp(vp_df, grouped_df):
    # build the lookup dict exactly as before
    subinstitution_dict = grouped_df.set_index(['Institution', 'SubInstitution'])['Category'].to_dict()

    # work on a copy so we don’t clobber the original unintentionally
    df = vp_df.copy()

    # prepare the two new helper columns
    df['PositionTitle'] = df['Position'].str.title()
    # .rfind returns -1 when not found, +14 yields e.g. 13 if 'Vice President' ends at idx -1+14
    df['vp_index'] = df['Position'].str.rfind('Vice President') + 14

    # pull in the Category for each (Institution, SubInstitution) pair
    # by mapping the multi-index tuples to subinstitution_dict
    df['Category'] = df.set_index(['Institution', 'SubInstitution']).index.map(subinstitution_dict)

    # mask A: "SubInstitution" or "Other" in FixedPosition,
    # subinst not NaN, and vp_index matches full length of PositionTitle
    mask_a = (
        df['FixedPosition'].str.contains('SubInstitution|Other', na=False) &
        df['SubInstitution'].notna() &
        (df['vp_index'] == df['PositionTitle'].str.len())
    )
    # apply: if Category != "Error" then Category else "Generic"
    df.loc[mask_a, 'FixedPosition'] = df.loc[mask_a, 'Category'].where(
        df.loc[mask_a, 'Category'] != 'Error',
        other='Generic'
    )

    # mask B: still contains "SubInstitution", but vp_index ≠ full length
    mask_b = (
        df['FixedPosition'].str.contains('SubInstitution', na=False) &
        (df['vp_index'] != df['PositionTitle'].str.len())
    )
    df.loc[mask_b, 'FixedPosition'] = df.loc[mask_b, 'PositionTitle'].apply(
        assign_position_grouping_subinst
    )

    # mask C: vp_index ≠ full length, SubInstitution is NaN, and "Other" in FixedPosition
    mask_c = (
        (df['vp_index'] != df['PositionTitle'].str.len()) &
        df['SubInstitution'].isna() &
        df['FixedPosition'].str.contains('Other', na=False)
    )
    def classify_or_other(pos):
        c = assign_position_grouping_subinst(pos)
        return c if c else 'Other'
    df.loc[mask_c, 'FixedPosition'] = df.loc[mask_c, 'PositionTitle'].apply(classify_or_other)

    # final override: anywhere FixedPosition is "Other", try assign_position… again
    mask_other = df['FixedPosition'] == 'Other'
    df.loc[mask_other, 'FixedPosition'] = df.loc[mask_other, 'PositionTitle'].apply(classify_or_other)

    # drop helpers if you like
    df = df.drop(columns=['PositionTitle', 'vp_index', 'Category'])

    return df

    

def last_sweep(df):
    out = df.copy()
    out['FixedPosition'] = out['FixedPosition'].fillna("").replace("", "Other")
    return out

def add_seniority(vp_df: pd.DataFrame) -> pd.DataFrame:
    df = vp_df.copy()

    pos_lc = df['Position'].fillna("").str.lower()
    df['Seniority'] = "Default"
    mask = pos_lc.str.contains(r"\bassistant\b")
    df.loc[mask, 'Seniority'] = "Assistant"

    mask = pos_lc.str.contains(r"\bassociate\b")
    df.loc[mask, 'Seniority'] = "Associate"

    mask = pos_lc.str.contains(r"\bsenior\b")
    df.loc[mask, 'Seniority'] = "Senior"

    mask = pos_lc.str.contains(r"\bexecutive\b")
    df.loc[mask, 'Seniority'] = "Executive"

    return df


In [20]:
data_path = r"C:\Projects\connecteddatahub\data"
years = ["1999","2000","2005","2007","2008","2009","2010","2011","2013","2018"]

grouped_validated_path = os.path.join(data_path, 'cleaned_dataframes', 'subinstitutions', 'full_validation.csv')
grouped_df = pd.read_csv(grouped_validated_path)

for year in years:
    split_path = os.path.join(data_path,'cleaned_dataframes',f'{year}_cleanedDataframe.csv')
    full_df = pd.read_csv(split_path)
    if 'Seniority' not in full_df.columns:
        full_df['Seniority'] = ""
    if 'Designation' not in full_df.columns:
        full_df['Designation'] = ""
    full_df['Position'] = full_df['Position'].fillna('')
    full_df['FixedPosition'] = full_df.get('FixedPosition', '')
    vp_mask = full_df['Position'].str.contains(r'Vice President',case=False, na=False)

    vp_df = full_df.loc[vp_mask].copy()
    vp_df = classify_vp(vp_df)
    vp_df = remaining_classification(vp_df)
    vp_df = classify_subinstitution_vp(vp_df, grouped_df)
    vp_df = last_sweep(vp_df)
    vp_df = add_seniority(vp_df)

    # prepend the “Vice President, ” prefix
    # vp_df['FixedPosition'] = "Vice President, " + vp_df['FixedPosition']
    vp_df['Designation']    = vp_df['FixedPosition']
    vp_df['FixedPosition']  = "Vice President"
    full_df.loc[vp_mask, ['FixedPosition','Designation']] = vp_df[['FixedPosition','Designation']]

    full_df.loc[vp_mask, 'FixedPosition'] = vp_df['FixedPosition']
    full_df.loc[vp_mask, 'Seniority'] = vp_df['Seniority']

    vp_path = os.path.join(data_path, 'cleaned_dataframes', 'vps', f'{year}_vice_presidents.csv')
    vp_df.to_csv(vp_path, index=False)

    full_path = os.path.join(data_path, 'cleaned_dataframes', f'{year}_cleanedDataframe.csv')
    full_df.to_csv(full_path, index = False)
